In [13]:
import pandas as pd

listings = pd.read_csv('./data/listings.csv')
reviews = pd.read_csv('./data/reviews.csv')

In [14]:
reviews.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

## Reviewer

In [15]:
reviewer = pd.DataFrame()
reviewer['reviewer_id'] = reviews['reviewer_id']
reviewer['reviewer_name'] = reviews['reviewer_name']

In [71]:
reviewer_final = reviewer.drop_duplicates(subset=['reviewer_id'])
reviewer_final = reviewer_final.reset_index(drop=True)

In [72]:
reviewer_final.to_csv('./data_sql/reviewer.csv', index=False, sep=';')

## Date

In [17]:
from datetime import datetime

def get_day(date): 
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.day

def get_month(date):
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.month

def get_year(date):
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.year


In [38]:
date = pd.DataFrame() 
date['date'] = reviews['date']

In [42]:
#drop duplicates
date = date.drop_duplicates()
date = date.reset_index(drop=True)

In [ ]:
#foreign key for day

size_date = len(date.index)
date['day_id'] = [i for i in range(size_date)]

for i in date['date'].index:
    date['day_id'][i] = get_day(date['date'][i])

In [ ]:
#foreign key for month

size_date = len(date.index)
date['month_id'] = [i for i in range(size_date)]
for i in date['date'].index:
    date['month_id'][i] = get_month(date['date'][i])

In [52]:
# Table day
day = pd.DataFrame()
day['day_number'] = [i for i in range(1,32)]

In [53]:
# Table month
month = pd.DataFrame()
month['month_number'] = [i for i in range(1,13)]
month['month_name'] = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [54]:
max = date['date'].max()
min = date['date'].min()

print(max, min)

2023-09-11 2010-07-22


In [55]:
# Table year -> min_year(date) - max_year(date)
year = pd.DataFrame()
year['year_number'] = [i for i in range(2010,2024)]

In [ ]:
# Foreign key of year_id

size_date = len(date.index)
date['year_id'] = [i for i in range(size_date)]
for i in date['date'].index:
    year_number = get_year(date['date'][i])
    row_num = year[year['year_number']==year_number].index
    date['year_id'][i] = row_num[0]+1

In [62]:
day.to_csv('./data_sql/day.csv', sep=';', index=False)
month.to_csv('./data_sql/month.csv', sep=';', index=False)
year.to_csv('./data_sql/year.csv', sep=';', index=False)
date.to_csv('./data_sql/date.csv', sep=';', index=False)

In [69]:
date = date.reset_index(drop=True)

## Review

In [64]:
review = pd.DataFrame()
review['reviewer_id'] = reviews['reviewer_id']
review['review'] = reviews['comments']

In [82]:
def get_date_id(date_str):
    return date[date['date']==date_str].index[0]+1

In [86]:
size_review = len(review.index)
review['date_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['date_id'][i] = get_date_id(reviews['date'][i])

C:\Users\filip\AppData\Local\Temp\ipykernel_15204\753352024.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['date_id'][i] = get_date_id(reviews['date'][i])


## Availability

In [ ]:
# Availability 
# Coloquei o id do listing como id da availability, nao deve haver problema (podemos trocar)
selected_columns = ['id', 'availability_30', 'availability_60', 'availability_90', 'availability_365']

availability = listings[selected_columns].copy()

## Location

In [ ]:
country = pd.DataFrame({'country_id': 0, 'country': ['Portugal'], 'country_code': ['PT']})

# Create a new dataframe that came from listings that take the city

city = pd.DataFrame()


city["city"] = listings["neighbourhood_group_cleansed"].drop_duplicates().reset_index(drop=True)
city["city_id"] = city.index
city["country_id"] = 0

# Create a new dataframe that came from listings that take the neighbourhood

neighbourhood = pd.DataFrame()

# Extract the 'neighbourhood_cleansed' column and drop duplicates
neighbourhood['combined_neighbourhood'] = listings['neighbourhood_cleansed'].drop_duplicates().reset_index(drop=True)

# Assuming 'city_column' is the column in 'listings' containing city information
neighbourhood = pd.merge(neighbourhood, listings[['neighbourhood_cleansed', 'neighbourhood_group_cleansed']], 
                         left_on='combined_neighbourhood', right_on='neighbourhood_cleansed', how='left')

neighbourhood.rename(columns={'neighbourhood_group_cleansed': 'city_column'}, inplace=True)
neighbourhood.drop(['neighbourhood_cleansed'], axis=1, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)


# Split the combined neighborhood string into individual neighborhoods using both commas and " e "
neighbourhood['neighbourhood'] = neighbourhood['combined_neighbourhood'].str.split(',| e ')

# Explode the DataFrame, creating a new row for each neighborhood
neighbourhood = neighbourhood.explode('neighbourhood')

# Drop unnecessary columns and rename columns
neighbourhood.rename(columns={'city_column': 'city'}, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)
neighbourhood = pd.merge(neighbourhood, city, on="city", how='left')
neighbourhood.drop(['combined_neighbourhood', 'city', 'country_id'], axis=1, inplace=True)